In [1]:
# %%
import numpy as np
from scipy.spatial import Delaunay, minkowski_distance
import meshio
import pygmsh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mesh import Mesh
import scipy.sparse.linalg as spla
from icosphere import icosphere
from geometry import project_vector_to_plane
#with pygmsh.geo.Geometry() as geom:
#    geom.add_rectangle(0.0, 1.0, 0.0, 1.0, 0.0, mesh_size=0.01)
#    mesh = geom.generate_mesh()

#points = mesh.points
#simplices = mesh.cells_dict['triangle']
#point_normals = np.array([[0,0,1]]*len(simplices))
#mesh = Mesh(points, simplices, point_normals)

points, simplices = icosphere(120)
point_normals = points / np.linalg.norm(points,axis=-1,keepdims=True)
print(len(simplices))
mesh = Mesh(points, simplices, point_normals)

if len(simplices) < 5000:
    # Setup for a 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot each simplex
    for simplex in simplices:
        polygon = points[simplex]
        ax.add_collection3d(Poly3DCollection([polygon], facecolors='grey', linewidths=1, edgecolors='k', alpha=.1))

    # Set plot display parameters
    ax.scatter(points[:,0], points[:,1], points[:,2], color='k')  # Plot the points
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # Adjusting the scale for better visualization
    max_range = np.array([points[:,0].max()-points[:,0].min(), 
                        points[:,1].max()-points[:,1].min(), 
                        points[:,2].max()-points[:,2].min()]).max() / 2.0
    mid_x = (points[:,0].max()+points[:,0].min()) * 0.5
    mid_y = (points[:,1].max()+points[:,1].min()) * 0.5
    mid_z = (points[:,2].max()+points[:,2].min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)

    plt.show()

288000


In [2]:
# %% Init
# \phi = x^2 + y^2 + (xy)^2
xc = mesh.barycenters[:,0]
yc = mesh.barycenters[:,1]
zc = mesh.barycenters[:,2]

phi = 1.0/np.cosh(100*(xc - 0.5))/np.cosh(100*(yc - 0.5))

xf = mesh.edge_centers[:,0]
yf = mesh.edge_centers[:,1]
zf = mesh.edge_centers[:,2]

u = np.random.randn(*(len(xc),3))

for i in range(len(xc)):
    u[i,:] = project_vector_to_plane(u[i,:],mesh.barynormals[i,:])

Iv = mesh.identity_matrix()

with meshio.xdmf.TimeSeriesWriter("burgers_test0.xdmf") as writer:
    writer.write_points_cells(points, [("triangle", simplices),])
    # Find face flux
    uf = mesh.interpolate_field_cell_to_face(u)
    un = np.einsum("ij,ij->i",uf, mesh.edge_normals)
    for _ in range(1):
        un, du = mesh.helmholtz_projection(un)
    writer.write_data(0, cell_data={"u": [u],"du": [du], "normals": [mesh.barynormals]})

In [3]:
with meshio.xdmf.TimeSeriesWriter("burgers_test.xdmf") as writer:
    writer.write_points_cells(points, [("triangle", simplices),])
    for t in range(201):
        # Solve
        dt = 0.02
        crank_nicolson_coeff = 0.5

        # Find face flux
        uf = mesh.interpolate_field_cell_to_face(u)
        un = np.einsum("ij,ij->i",uf, mesh.edge_normals)
        for _ in range(1):
            un, du = mesh.helmholtz_projection(un)
            u += du
        uf = un[:, np.newaxis] * mesh.edge_normals
        Jx,rhs_x = mesh.convection_matrix(u[:,0], uf, face_flux=True, quick=True)
        Jy,rhs_y = mesh.convection_matrix(u[:,1], uf, face_flux=True, quick=True)
        Jz,rhs_z = mesh.convection_matrix(u[:,2], uf, face_flux=True, quick=True)

        u[:,0] = spla.gmres(Iv+crank_nicolson_coeff*dt*Jx, u[:,0]*mesh.areas-(1.0-crank_nicolson_coeff)*dt*Jx@u[:,0]+dt*rhs_x)[0]
        u[:,1] = spla.gmres(Iv+crank_nicolson_coeff*dt*Jy, u[:,1]*mesh.areas-(1.0-crank_nicolson_coeff)*dt*Jy@u[:,1]+dt*rhs_y)[0]
        u[:,2] = spla.gmres(Iv+crank_nicolson_coeff*dt*Jz, u[:,2]*mesh.areas-(1.0-crank_nicolson_coeff)*dt*Jz@u[:,2]+dt*rhs_z)[0]
        for i in range(len(xc)):
            u[i,:] = project_vector_to_plane(u[i,:],mesh.barynormals[i,:])
        print(np.max(np.abs(u)))
        
        
        print(t*dt)
        writer.write_data(t*dt, cell_data={"u": [u]})

3.336425648029894
0.0
2.2132330515393575
0.02
1.4435517215292468
0.04
1.0930507129577163
0.06
0.9176375396764106
0.08
0.8553067790108335
0.1
0.7582586624333882
0.12
0.6690173556840582
0.14
0.6158896730926113
0.16
0.578538994227531
0.18
0.5764104621803271
0.2
0.5173926078552611
0.22
0.510709425815248
0.24
0.5081718988644509
0.26
0.49246824617896306
0.28
0.4871750674153289
0.3
0.48056034505931144
0.32
0.4682351953625674
0.34
0.45128560278658053
0.36
0.44223594146460554
0.38
0.44149991867369887
0.4
0.4334994199143624
0.42
0.42430641341345915
0.44
0.4094497416815491
0.46
0.40367992270598624
0.48
0.4033915487400548
0.5
0.39470690403486275
0.52
0.3876466169179962
0.54
0.38543461179963706
0.56
0.38259058083266206
0.58
0.37899258713218126
0.6
0.3762907101511648
0.62
0.37513808563561624
0.64
0.3749240321659992
0.66
0.374268454835938
0.68
0.37504383515423456
0.7000000000000001
0.3719922442384511
0.72
0.3715849673749996
0.74
0.37209269412061335
0.76
0.37328149673322264
0.78
0.3736506190558898
0.8